# Homework 4 - Stock Data Analysis

## Problem 1 - IBM Stock Analysis

In [35]:
# Importing data
import pandas as pd

ibm_df = pd.read_csv('ibm-1.csv')
ibm_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,28-Dec-18,109.20,109.75,107.55,108.06,82.36,"5,352,905"
1,27-Dec-18,105.15,108.78,104.66,108.78,82.91,"6,323,698"
2,26-Dec-18,103.25,106.49,101.28,106.49,81.16,"6,951,507"
3,24-Dec-18,105.07,106.12,102.77,102.84,78.38,"3,997,184"
4,21-Dec-18,107.55,110.21,105.58,106.06,80.84,"10,864,279"


In [36]:
# Clean data
ibm_df['Date'] = pd.to_datetime(ibm_df['Date'], format='%d-%b-%y')
ibm_df['Volume'] = ibm_df['Volume'].str.replace(',', '').astype(int)
for col in ['Open', 'High', 'Low', 'Close']:
    ibm_df[col] = ibm_df[col].astype(float)
ibm_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-12-28,109.20,109.75,107.55,108.06,82.36,5352905
1,2018-12-27,105.15,108.78,104.66,108.78,82.91,6323698
2,2018-12-26,103.25,106.49,101.28,106.49,81.16,6951507
3,2018-12-24,105.07,106.12,102.77,102.84,78.38,3997184
4,2018-12-21,107.55,110.21,105.58,106.06,80.84,10864279


In [37]:
# Calculating trade value
ibm_df['Trade Value'] = ibm_df['Close'] * ibm_df['Volume']
ibm_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Trade Value
0,2018-12-28,109.20,109.75,107.55,108.06,82.36,5352905,5.784349e+08
1,2018-12-27,105.15,108.78,104.66,108.78,82.91,6323698,6.878919e+08
2,2018-12-26,103.25,106.49,101.28,106.49,81.16,6951507,7.402660e+08
3,2018-12-24,105.07,106.12,102.77,102.84,78.38,3997184,4.110704e+08
4,2018-12-21,107.55,110.21,105.58,106.06,80.84,10864279,1.152265e+09


In [38]:
# Close price statistics
highest_close = ibm_df['Close'].max()
average_close = ibm_df['Close'].mean()
print('The average closing price was ${0:.2f} with a high of ${1:.2f}'.format(average_close, highest_close))

The average closing price was $137.64 with a high of $161.68


In [39]:
# Days above average close
above_avg = ibm_df[ibm_df['Close'] > average_close]
above_avg.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Trade Value
50,2018-10-16,136.64,139.05,135.35,138.74,104.41,9095493,1.261909e+09
55,2018-10-09,141.57,141.84,140.47,140.76,105.94,3809950,5.362886e+08
56,2018-10-08,141.97,142.51,140.71,141.86,106.76,3388308,4.806654e+08
57,2018-10-05,144.26,145.11,141.37,142.48,107.23,4813901,6.858846e+08
58,2018-10-04,145.02,146.08,143.75,144.66,108.87,4432843,6.412551e+08


In [44]:
# Convert to Japanese Yen
yen_per_usd = 107
yen_df = ibm_df.copy()

for col in ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Trade Value']:
    yen_df[col] = yen_df[col] * yen_per_usd

# Display in yen format but keep numeric values
yen_df.head().style.format(
    {col: "¥{:,.2f}" for col in ['Open','High','Low','Close','Adj Close','Trade Value']} |
    {'Date': "{:%d-%b-%y}"}
)


,Date,Open,High,Low,Close,Adj Close,Volume,Trade Value
0,28-Dec-18,"¥11,684.40","¥11,743.25","¥11,507.85","¥11,562.42","¥8,812.52",5352905,"¥61,892,535,830.10"
1,27-Dec-18,"¥11,251.05","¥11,639.46","¥11,198.62","¥11,639.46","¥8,871.37",6323698,"¥73,604,429,923.08"
2,26-Dec-18,"¥11,047.75","¥11,394.43","¥10,836.96","¥11,394.43","¥8,684.12",6951507,"¥79,208,459,906.01"
3,24-Dec-18,"¥11,242.49","¥11,354.84","¥10,996.39","¥11,003.88","¥8,386.66",3997184,"¥43,984,533,073.92"
4,21-Dec-18,"¥11,507.85","¥11,792.47","¥11,297.06","¥11,348.42","¥8,649.88",10864279,"¥123,292,401,089.18"
